In [ ]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.4/982.4 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.3 MB/s eta 0:00:00


# Tracking usando cálculo do centro de massa

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

# Carregar o modelo treinado
model = YOLO("best.pt")

# Vídeo de entrada e saída
input_video_path = "tracking_video_test.mp4"
output_video_path = "tracking_video_test_tracked.mp4"

# Abrir vídeo
cap = cv2.VideoCapture(input_video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# Lista de trajetórias (uma lista para cada animal detectado por ordem)
trajectories = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Inferência
    results = model(frame, conf=0.3, imgsz=640)

    # Copiar frame original para desenhar
    frame_output = frame.copy()

    # Resetar trajetórias por frame
    current_centers = []

    if results[0].masks is not None:
        masks = results[0].masks.data.cpu().numpy()  # Shape: (N, H, W)

        for i, mask in enumerate(masks):
            # Binarizar a máscara e redimensionar
            mask_bin = (mask * 255).astype(np.uint8)
            mask_bin = cv2.resize(mask_bin, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_NEAREST)

            # Desenhar máscara sobre o frame
            colored_mask = np.zeros_like(frame)
            colored_mask[mask_bin > 0] = (0, 255, 0)  # Verde
            frame_output = cv2.addWeighted(frame_output, 1.0, colored_mask, 0.4, 0)

            # Calcular centro de massa
            M = cv2.moments(mask_bin)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                current_centers.append((cx, cy))

                # Expandir lista de trajetórias se necessário
                if len(trajectories) <= i:
                    trajectories.append([])

                trajectories[i].append((cx, cy))

                # Desenhar ponto central
                cv2.circle(frame_output, (cx, cy), 5, (255, 0, 0), -1)
                cv2.putText(frame_output, f"ID {i+1}", (cx + 10, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    # Desenhar trajetórias acumuladas
    for traj in trajectories:
        for i in range(1, len(traj)):
            if traj[i - 1] and traj[i]:
                cv2.line(frame_output, traj[i - 1], traj[i], (0, 0, 255), 2)

    out.write(frame_output)

cap.release()
out.release()
print(f"Tracking salvo em: {output_video_path}")


# Tracking usando o YOLO

## Código 1 documentação (Traçar trajectos em vários fotogramas de vídeo)

In [ ]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("best.pt")

# Open the video file
video_path = "video_teste7.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties for VideoWriter
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
output_path = "output_video_7.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if not success:
        break  # Exit the loop if no more frames

    # Run YOLO tracking on the frame, persisting tracks between frames
    result = model.track(frame, persist=True)[0]

    # Get the boxes and track IDs
    if result.boxes and result.boxes.id is not None:
        boxes = result.boxes.xywh.cpu()
        track_ids = result.boxes.id.int().cpu().tolist()

        # Visualize the result on the frame
        frame = result.plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 30 tracks for 30 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

    # Write the frame to the output video
    out.write(frame)

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video saved to {output_path}")


0: 384x640 2 cows, 56.7ms
Speed: 2.7ms preprocess, 56.7ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cows, 50.1ms
Speed: 3.4ms preprocess, 50.1ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cows, 43.8ms
Speed: 3.4ms preprocess, 43.8ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cows, 43.5ms
Speed: 3.3ms preprocess, 43.5ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cows, 41.7ms
Speed: 4.6ms preprocess, 41.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cows, 40.9ms
Speed: 3.3ms preprocess, 40.9ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cows, 41.7ms
Speed: 3.4ms preprocess, 41.7ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cows, 41.1ms
Speed: 3.3ms preprocess, 41.1ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x